## 1. セットアップ

In [1]:
import sys
from pprint import pprint
sys.path.append("/home/paper_translator/")
pprint(sys.path)

['/home/paper_translator/lib',
 '/home/paper_translator',
 '/usr/lib/python311.zip',
 '/usr/lib/python3.11',
 '/usr/lib/python3.11/lib-dynload',
 '',
 '/home/paper_translator/.venv/lib/python3.11/site-packages',
 '/home/paper_translator/']


In [2]:
import logging
import sys

# ログレベルの設定
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, force=True)

In [3]:
from llama_index.callbacks import CallbackManager, LlamaDebugHandler
llmama_debug_handler = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llmama_debug_handler])

INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


## 2. ドキュメントの準備

In [4]:
from src.XMLUtils import DocumentCreator

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/home/paper_translator/.venv/lib/python3.11/site-packages/certifi/cacert.pem'
DEBUG:httpx:load_verify_locations cafile='/home/paper_translator/.venv/lib/python3.11/site-packages/certifi/cacert.pem'
DEBUG:slack_bolt.App:Sending a request - url: https://www.slack.com/api/auth.test, query_params: {}, body_params: {}, files: {}, json_body: None, headers: {'Content-Type': 'application/x-www-form-urlencoded', 'Authorization': '(redacted)', 'User-Agent': 'Bolt/1.18.0 Python/3.11.5 slackclient/3.23.0 Linux/5.15.133.1-microsoft-standard-WSL2'}
DEBUG:slack_bolt.App:Received the following response - status: 200, headers: {'date': 'Tue, 24 Oct 2023 22:14:44 GMT', 'server': 'Apache', 'vary': 'Accept-Encoding', 'x-slack-req-id': 'a20c069657fea01a72887dcea7109d12', 'x-content-type-options': 'nosniff', 'x-xss-protection': '0', 'pragma': 'no-cache', 'cache-control': 'private, no-cache

In [5]:
base_path = "/home/paper_translator/data"
document_name = (
    "Learning_Transferable_Visual_Models_From_Natural_Language_Supervision"
    #"ChatKBQA_A_Generate-then-Retrieve_Framework_for_Knowledge_Base_Question_Answering_with_Fine-tuned_Large_Language_Models"
)
document_path = f"{base_path}/documents/{document_name}"
xml_path = f"{document_path}/{document_name}.tei.xml"

In [6]:
# 自作の DirectoryReader を使用して、
# ディレクトリ内の xml ファイルをDocumentオブジェクトとして読み込む
# run_grobid(dir_path, pdf_name)
creator = DocumentCreator()
creator.load_xml(xml_path, contain_abst=False)
docs = creator.create_docs()
print(f"documents_2 metadata: \n{docs[0].metadata}")
print(f"documents_2 text: \n{docs[0].text}")

documents_2 metadata: 
{'Section No.': '1.', 'Section Title': 'Introduction and Motivating Work', 'Title': 'Learning Transferable Visual Models From Natural Language Supervision', 'Authors': 'Alec Radford, Jong Wook Kim, Chris Hallacy, Aditya Ramesh, Gabriel Goh, Sandhini Agarwal, Girish Sastry, Amanda Askell, Pamela Mishkin, Jack Clark, Gretchen Krueger, Ilya Sutskever', 'Idno': 'arXiv:2103.00020v1[cs.CV]', 'Published': '26 Feb 2021', 'Language': 'en'}
documents_2 text: 
Pre-training methods which learn directly from raw text have revolutionized NLP over the last few years (Dai & Le, 2015;Peters et al., 2018;Howard & Ruder, 2018;Radford et al., 2018;Devlin et al., 2018;Raffel et al., 2019).Task-agnostic objectives such as autoregressive and masked language modeling have scaled across many orders of magnitude in compute, model capacity, and data, steadily improving capabilities. The development of "text-to-text" as a standardized input-output interface (McCann et al., 2018;Radford et a

## 3. Context の設定

In [7]:
from src.translator.llama_cpp import create_llama_cpp_model

model_path = "/home/paper_translator/data/models/ELYZA-japanese-Llama-2-7b-fast-instruct-q4_K_M.gguf"
llm = create_llama_cpp_model(package_name="llama_index", model_path=model_path)

ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce GTX 1660 Ti, compute capability 7.5
llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /home/paper_translator/data/models/ELYZA-japanese-Llama-2-7b-fast-instruct-q4_K_M.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  4096, 45043,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q6_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q4_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    6:            blk.0.f

###  3.1. SentenceWindowNodeParser の設定

In [8]:
from llama_index.node_parser import SentenceWindowNodeParser

node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="sentence_window",
    original_text_metadata_key="original_text",
)

### 3.2. Embedding の設定

In [9]:
from llama_index.embeddings import HuggingFaceEmbedding

model_name = "sentence-transformers/all-MiniLM-l6-v2"
chunk_size = 3072

embed_model = HuggingFaceEmbedding(
    model_name=model_name, max_length=chunk_size, device="cuda:0"
)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443


DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-l6-v2/resolve/main/config.json HTTP/1.1" 307 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-l6-v2/resolve/main/tokenizer_config.json HTTP/1.1" 307 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/tokenizer_config.json HTTP/1.1" 200 0


### 3.3. ServiceContext の設定

In [10]:
from llama_index import ServiceContext

ctx = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
    node_parser=node_parser,
    callback_manager=callback_manager,
    chunk_size=chunk_size,
)

### 3.4. StorageContext の設定

In [11]:
from llama_index import StorageContext
from llama_index.storage.docstore import SimpleDocumentStore
from llama_index.storage.index_store import SimpleIndexStore
from llama_index.vector_stores import SimpleVectorStore

# Storage Context の作成
storage_context = StorageContext.from_defaults(
    docstore = SimpleDocumentStore(),
    vector_store = SimpleVectorStore(),
    index_store = SimpleIndexStore()
)

## 4. Index の作成

### 4.1. セットアップ

In [12]:
from llama_index.llms.base import ChatMessage, MessageRole
from llama_index.prompts import ChatPromptTemplate

In [28]:
# (1) QAプロンプトの定義。
# QAシステムプロンプト
TEXT_QA_SYSTEM_PROMPT = ChatMessage(
    content=(
            "#依頼\n"
            "あなたは高度な理解能力を持ち、複雑なテキストも簡潔に要約することができるAIです。\n"
            "事前知識ではなく、提供されたコンテキストに基づいて精確な回答を行ってください。\n"
            "#従うべきルール\n"
            "1. 略語や初出の用語には解説を加え、AI分野やコンピュータの初心者も理解できるように工夫してください。\n"
            "2. 回答内で指定されたコンテキストを直接参照しないでください。\n"
            "3. 「コンテキストに基づいて、...」や「コンテキスト情報は...」、またはそれに類するような記述は避けてください。\n"
            "#手順\n"
            "1. 与えられたコンテキストに含まれる主要なポイントやコンセプトを細かく分解してください。\n"
            "2. それぞれのポイントやコンセプトに対して詳細な説明を加えてください。\n"
            "3. まずは指示に従って、文書の初版を作成してください。\n"
            "4. 作成した初版をルールに従っているか自己分析してください。\n"
            "5. 自己分析の結果を踏まえて、文書を改善してください。\n"
    ),
    role=MessageRole.SYSTEM,
)

# QAプロンプトテンプレートメッセージ
TEXT_QA_PROMPT_TMPL_MSGS = [
    TEXT_QA_SYSTEM_PROMPT,
    ChatMessage(
        content=(
            "複数のソースからのコンテキスト情報を以下に示します。\n"
            "---------------------\n"
            "{context_str}\n"
            "---------------------\n"
            "予備知識ではなく、複数のソースからの情報を考慮して質問に答えてください。\n"
            "疑問がある場合は、「情報無し」と答えてください。\n"
            "Query: {query_str}\n"
            "Answer: "
        ),
        role=MessageRole.USER,
    ),
]

# チャットQAプロンプト
CHAT_TEXT_QA_PROMPT = ChatPromptTemplate(message_templates=TEXT_QA_PROMPT_TMPL_MSGS)

In [29]:
# (2) TreeSummarizeプロンプトの定義。
# QAシステムプロンプト
TEXT_QA_SYSTEM_PROMPT = ChatMessage(
    content=(
            "#依頼\n"
            "あなたは高度な理解能力を持ち、複雑なテキストも簡潔に要約することができるAIです。\n"
            "事前知識ではなく、提供されたコンテキストに基づいて精確な回答を行ってください。\n"
            "#従うべきルール\n"
            "1. 略語や初出の用語には解説を加え、AI分野やコンピュータの初心者も理解できるように工夫してください。\n"
            "2. 回答内で指定されたコンテキストを直接参照しないでください。\n"
            "3. 「コンテキストに基づいて、...」や「コンテキスト情報は...」、またはそれに類するような記述は避けてください。\n"
            "#手順\n"
            "1. 与えられたコンテキストに含まれる主要なポイントやコンセプトを細かく分解してください。\n"
            "2. それぞれのポイントやコンセプトに対して詳細な説明を加えてください。\n"
            "3. まずは指示に従って、文書の初版を作成してください。\n"
            "4. 作成した初版をルールに従っているか自己分析してください。\n"
            "5. 自己分析の結果を踏まえて、文書を改善してください。\n"
    ),
    role=MessageRole.SYSTEM,
)

# ツリー要約プロンプトメッセージ
TREE_SUMMARIZE_PROMPT_TMPL_MSGS = [
    TEXT_QA_SYSTEM_PROMPT,
    ChatMessage(
        content=(
            "複数のソースからのコンテキスト情報を以下に示します。\n"
            "---------------------\n"
            "{context_str}\n"
            "---------------------\n"
            "予備知識ではなく、複数のソースからの情報を考慮して質問に答えてください。\n"
            "疑問がある場合は、「情報無し」と答えてください。\n"
            "Query: {query_str}\n"
            "Answer: "
        ),
        role=MessageRole.USER,
    ),
]

# ツリー要約プロンプト
CHAT_TREE_SUMMARIZE_PROMPT = ChatPromptTemplate(
    message_templates=TREE_SUMMARIZE_PROMPT_TMPL_MSGS
)

In [30]:
# Summaryクエリ
SUMMARY_QUERY = "提供されたテキストの内容を要約してください。"

In [31]:
import nest_asyncio
from llama_index import get_response_synthesizer

# 非同期処理の有効化
nest_asyncio.apply()

# レスポンスシンセサイザーの準備
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
    use_async=True,
    text_qa_template=CHAT_TEXT_QA_PROMPT,  # QAプロンプト
    summary_template=CHAT_TREE_SUMMARIZE_PROMPT,  # TreeSummarizeプロンプト
)

### 4.2. DocumentSummaryIndex の作成

In [32]:
from llama_index.indices.document_summary import DocumentSummaryIndex

# DocumentSummaryIndexの準備
doc_summary_index = DocumentSummaryIndex.from_documents(
    docs,
    storage_context=storage_context,
    service_context=ctx,
    response_synthesizer=response_synthesizer,
    summary_query=SUMMARY_QUERY,  # 要約クエリ
)

DEBUG:llama_index.node_parser.node_utils:> Adding chunk: Pre-training methods which learn directly from ...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: The development of "text-to-text" as a standard...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: Flagship systems like GPT-3 (Brown et al., 2020...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: However, in other fields such as computer visio...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: Could scalable pre-training methods which learn...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: Prior work is encouraging.Over 20 years ago Mor...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: (1999) explored improving content based image r...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: Quattoni et al. 
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: (2007) demonstrated it was possible to learn mo...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: Sriva

KeyboardInterrupt: 

In [18]:
for i in range(len(doc_summary_index.index_id)):
    print(f"{i}")
    try:
        print(doc_summary_index.get_document_summary(f"{i}"))
    except ValueError:
        print(f"doc_id: {i} is no text.")
        continue

0
提供されたテキストは、Alec Radford、Jong Wook Kim、Chris Hallacy、Aditya Ramesh、Gabriel Goh、Sandhini Agarwal、Girish Sastry、Amanda Askell、Pamela Mishkin、Jack Clark、Gretchen Krueger、Ilya Sutskeverによって書かれた「Learning Transferable Visual Models From Natural Language Supervision」というタイトルの論文の一部です。この論文では、自然言語の監督から転移可能なビジュアルモデルの学習について説明されています。また、NLPの分野では直接生のテキストから学習する事前学習方法が革新的な成果を上げている一方、コンピュータビジョンの分野では従来通りクラウドラベル付きのデータセットでモデルを事前学習するのが一般的であることが指摘されています。さらに、過去の研究や実験結果も紹介されています。
1
doc_id: 1 is no text.
2
提供されたテキストは、Alec Radfordらによる「Learning Transferable Visual Models From Natural Language Supervision」というタイトルの論文の一部です。この論文では、自然言語に含まれる監督信号から知覚を学習するアプローチについて説明されています。また、他の研究者による関連研究も言及されており、自然言語からの学習が他のトレーニング方法よりもいくつかの利点を持つことが強調されています。具体的なアプローチについては、後続のサブセクションで詳細が説明されています。
3
提供されたテキストによれば、Alec Radfordらによる「Learning Transferable Visual Models From Natural Language Supervision」というタイトルの論文があります。この論文では、既存のデータセットが十分な大きさを持っていないことが指摘されており、MS-COCO、Visual Genome、YFCC100Mという3つのデータセットが主に使用されていることが述べられています。しかし、これらのデータセットは現代の基準で